此文件用于转换图像数据格式，将.nii.gz和.dcm文件转换为.npy文件

In [1]:
# 基础库
import numpy as np
import os
# 图像.dcm文件读取
import pydicom
# 标签.nii.gz文件读取
import nibabel as nib

In [2]:
# .nii.gz文件转array
def nii2array(nii_path):
    img = nib.load(nii_path)
    img_array = np.asarray(img.get_fdata())
    img_array = np.transpose(img_array, (2, 1, 0))  # (x, y, z)
    img_array = np.around(img_array)
    img_array = img_array.astype(np.int16)
    return img_array

In [3]:
# .dcm文件转array
def dcm2array(dcm_path):
    slices = [pydicom.read_file(dcm_path + '/' + s) for s in os.listdir(dcm_path)]
    # 按照z轴的位置排序
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    lst = []
    for s in slices:
        print(s.pixel_array.shape)
        if(s.pixel_array.shape==(512,512)):
            lst.append(s.pixel_array)
    # 转换为np.array
    image = np.stack(lst)  # (z, y, x)
    #将外部补全的部分修改成空气的数值
    image[image < 0] = 0
    return image.astype(np.int16)

In [4]:
def main():
    save_path=  "./npy"
    data_path = "E:/main_file/项目/影像组学/临床信息核心数据/TCGA-KIRC/TCGA-B0-4713/06-22-1992-NA-ct ap wwo-47864/4.000000-Recon 2 WITH AND DELAY-53536/"
    # ds = pydicom.dcmread(data_path+"1-084.dcm")
    # data_array= np.array(ds.pixel_array)
    data_array = dcm2array(data_path)
    print(data_array.shape)
    np.save(save_path + "final" + ".npy",
             data_array.reshape(1, data_array.shape[0], data_array.shape[1], data_array.shape[2]))
    print(data_array)

In [5]:
if __name__ == "__main__":
    main()

(512, 512)
[[-1024 -1024 -1024 ... -1024 -1024 -1024]
 [-1024 -1024 -1024 ... -1024 -1024 -1024]
 [-1024 -1024 -1024 ... -1024 -1024 -1024]
 ...
 [-1024 -1024 -1024 ... -1024 -1024 -1024]
 [-1024 -1024 -1024 ... -1024 -1024 -1024]
 [-1024 -1024 -1024 ... -1024 -1024 -1024]]
